In [2]:
import csv
import numpy as np
import pickle
import os
from datetime import datetime
from collections import Counter
from tqdm import tqdm

In [3]:
def one_hot(data, ret_type):
    #print(data)
    counts = Counter(data)
    output = []
    
    if ret_type == "coin":
        values = {"ETH": 0, "WETH": 1, "WAX": 2}
        
    elif ret_type == "market":
        values = {"OpenSea": 0, "Atomic": 1, "Cryptokitties": 2, "Godsunchained": 3, "Decentraland": 4}
        
    else:
        values = {"Other": 0, "Art": 1, "Games": 2, "Collectible": 3}
        
    for v in values.keys():
        if v not in counts.keys():
            output.append(0)
        else:
            output.append(counts[v])
    #print(output)
    return output

In [14]:
def loadCollection(c):
    return pickle.load(open(f"{DIR}{c}", "rb"))

def getTimestamp(date):
    return datetime.fromisoformat(date).timestamp()

def getCollectionDateRange(c_nfts):
    dates = {i: getTimestamp(el[19]) for i, el in enumerate(c_nfts)}
    return c_nfts[max(dates, key=dates.get)][19], c_nfts[min(dates, key=dates.get)][19]

def collStats(c, start, end):
    doc_vars = [[]]*6
    #start_ts = getTimestamp(start)
    #end_ts = getTimestamp(end)
    
    c_nfts = loadCollection(c)
    #print(c_nfts)
    #maxDate, minDate = getCollectionDateRange(c_nfts)
    
    for nft in c_nfts:
        date = nft[19]
        date_ts = getTimestamp(date)
        if date_ts >= start and date_ts <= end:
            doc_vars[0] = doc_vars[0] + [nft[13]] # USD
            doc_vars[1] = doc_vars[1] + [nft[11]] # CRYPTO
            doc_vars[2] = doc_vars[2] + [nft[12]] # COIN
            #doc_vars[3] = doc_vars[3] + [nft[5]]  # Buyer address
            doc_vars[3] = doc_vars[3] + [nft[17]] # Market
            doc_vars[4] = doc_vars[4] + [nft[23]] # Category
    
    output = []
    for el in doc_vars[1:2]:
        #print(el)
        el = np.array(el).astype(float)
        if len(el) > 0:
            output = output + [np.mean(el), np.median(el), np.var(el), np.sum(el), np.prod(el)]
        else: output = output + [0]*5
    output = output + one_hot(doc_vars[2], "coin")
    output = output + one_hot(doc_vars[3], "market")
    output = output + one_hot(doc_vars[4], "category")
    output.append(len(c_nfts))
    return output

In [15]:
DIR = "/media/hwixley/data-drive/mlp-coursework3/OSF-data/"
window_size = 10

In [16]:
files = os.listdir(f"{DIR}windows/size{window_size}")
print(files)

['Pandemicgame', 'Davincicollection', 'Mddeesignerr', 'Demonicgirlz', 'Hedbighedbig', 'Licarthen', 'Illumination', 'Myownarts', 'Ethernitywtti', 'Nowthenwhencollection', 'Zodiac', 'Sticknshits', 'Paigedragon', 'Crufromedia', 'Dejavu', 'Hiphophotte', 'Wpcwrarecard', 'Japanmechadv', 'Viruse', 'Superclstyle', 'Mrmonkpropaganda', 'Wivfuture', 'Ligermanspor', 'Hugamonster', 'Greenrabbit', 'Jyglitch', 'Sandiamaeu', 'Petrdavydtchenko', 'Xuniverse', 'Fndmusqqqq', 'Rplanatverif', 'Pornhubcard', 'Julianedelmanincredelmannftcollection', 'Cryptoshirt', 'Alwayspastel', 'Acryptocoin', 'Toltecwarrio', 'Folk', 'Cryptocross', 'Surrogatesofficial', 'Powpanda', 'Brianromero', 'Blobsedition', 'Thiccpokemon', 'Azurineart', 'Parody', 'Graffiti', 'Absurd', 'Unifty', 'Hmsequencing', 'Fedfedaxie', 'Unitedartist', 'Lapin', 'Cryptolighter', 'Gradgrid', 'Aeforia', 'Perrycper', 'Spiral', 'Eyescollection', 'Japanesetraditionalcolor', 'Crative', 'Sighbster', 'Cosantacosan', 'Banillaxtractsltd', 'Pinupart', 'Ethday',

In [18]:
window_size = 10
# interval: crypto, time_difference
all_samples = []
all_labels = []

for f in tqdm(files):
    samples = pickle.load(open(f"{DIR}windows/size{window_size}/{f}", "rb"))
    #print(samples)
    for s in tqdm(samples):
        #s = s[0]
        if len(s) > 0 and len(s[0]) > 0:
            s = s[0]
            sample_size = int((len(s) - 2)/3)
            assert sample_size == 10
            
            labels = s[-2:]
            s = s[:-2]
            start = s[2]
            end = s[-3]
            
            coll_stats = collStats(f, start, end)
            #print(coll_stats)
            x = [[s[i*3], s[i*3 + 1]] for i in range(sample_size)]
            #print(x)
            x = [i for el in x for i in el]
            x = x + coll_stats
            
            #print(x)
            all_samples.append(x)
            all_labels.append(labels)

100%|█████████████████████████████████████████| 6/6 [00:00<00:00, 125829.12it/s]

100%|███████████████████████████████████████| 13/13 [00:00<00:00, 436207.62it/s]

100%|███████████████████████████████████████| 17/17 [00:00<00:00, 552737.74it/s]

100%|███████████████████████████████████████| 25/25 [00:00<00:00, 738433.80it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 76260.07it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38836.15it/s]

100%|███████████████████████████████████████| 24/24 [00:00<00:00, 751218.63it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 109416.63it/s]

100%|███████████████████████████████████████| 19/19 [00:00<00:00, 510844.72it/s]

100%|███████████████████████████████████████| 13/13 [00:00<00:00, 432745.65it/s]

100%|███████████████████████████████████████| 24/24 [00:00<00:00, 774333.05it/s]

100%|█████████████████████████████████████████| 8/8 [00:00<00:00, 281970.02it/s]

100%|███████████

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 39568.91it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 76959.71it/s]

100%|███████████████████████████████████████| 20/20 [00:00<00:00, 660520.31it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 113359.57it/s]

100%|███████████████████████████████████████| 10/10 [00:00<00:00, 358487.52it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38836.15it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 113359.57it/s]

100%|████████████████████████████████████████| 19/19 [00:00<00:00, 46171.37it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38130.04it/s]

100%|██████████████████████████████████████| 67/67 [00:00<00:00, 1672728.38it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 39199.10it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38836.15it/s]

100%|███████████

100%|█████████████████████████████████████████| 96/96 [00:00<00:00, 1996.35it/s]

100%|████████████████████████████████████████| 11/11 [00:00<00:00, 30234.17it/s]

100%|███████████████████████████████████████| 74/74 [00:00<00:00, 139183.18it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 74235.47it/s]

100%|███████████████████████████████████████| 12/12 [00:00<00:00, 445412.81it/s]

100%|██████████████████████████████████████| 37/37 [00:00<00:00, 1149549.99it/s]

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3744.91it/s]

100%|████████████████████████████████| 45867/45867 [00:00<00:00, 4780343.44it/s]

100%|████████████████████████████████████| 833/833 [00:00<00:00, 4561168.71it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 58254.22it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 117597.31it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 42366.71it/s]

100%|███████████

100%|█████████████████████████████████████████| 12/12 [00:00<00:00, 8311.04it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 75573.05it/s]

100%|████████████████████████████████████| 244/244 [00:00<00:00, 3457466.81it/s]

100%|█████████████████████████████████████████| 8/8 [00:00<00:00, 294337.12it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 40721.40it/s]

100%|█████████████████████████████████████████| 8/8 [00:00<00:00, 294337.12it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 116508.44it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 37117.73it/s]

100%|████████████████████████████████████| 389/389 [00:00<00:00, 3821040.41it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 112347.43it/s]

100%|███████████████████████████████████████| 29/29 [00:00<00:00, 914547.49it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38130.04it/s]

100%|███████████

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 74235.47it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 36472.21it/s]

100%|██████████████████████████████████████| 43/43 [00:00<00:00, 1243828.08it/s]

100%|█████████████████████████████████████████| 4/4 [00:00<00:00, 153919.41it/s]

100%|█████████████████████████████████████████| 5/5 [00:00<00:00, 187245.71it/s]

100%|██████████████████████████████████████| 71/71 [00:00<00:00, 1793949.30it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 39568.91it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38479.85it/s]

100%|█████████████████████████████████████████| 5/5 [00:00<00:00, 145635.56it/s]

100%|███████████████████████████████████████| 10/10 [00:00<00:00, 352462.52it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 32263.88it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 70492.50it/s]

100%|███████████

...

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 36472.21it/s]

100%|███████████████████████████████████████| 14/14 [00:00<00:00, 383792.52it/s]

100%|██████████████████████████████████████| 51/51 [00:00<00:00, 1398101.33it/s]

100%|██████████████████████████████████████| 41/41 [00:00<00:00, 1154137.34it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 72315.59it/s]

100%|███████████████████████████████████████| 17/17 [00:00<00:00, 398341.72it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38479.85it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 26379.27it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 101475.10it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 110376.42it/s]

100%|██████████████████████████████████████| 55/55 [00:00<00:00, 1460042.53it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 37449.14it/s]

100%|███████████

100%|█████████████████████████████████████████| 4/4 [00:00<00:00, 144631.17it/s]

100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 6255.49it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38130.04it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 103991.01it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 109416.63it/s]

100%|█████████████████████████████████████████| 4/4 [00:00<00:00, 134217.73it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 37786.52it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 74235.47it/s]

100%|███████████████████████████████████████| 11/11 [00:00<00:00, 397735.72it/s]

100%|█████████████████████████████████████████| 8/8 [00:00<00:00, 189573.06it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 72944.42it/s]

100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 14588.88it/s]

100%|███████████

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 75573.05it/s]

100%|███████████████████████████████████████| 20/20 [00:00<00:00, 514638.53it/s]

100%|███████████████████████████████████████| 11/11 [00:00<00:00, 390994.44it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 59074.70it/s]

100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 89240.51it/s]

100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 75346.78it/s]

100%|██████████████████████████████████████| 43/43 [00:00<00:00, 1202367.15it/s]

100%|██████████████████████████████████| 9728/9728 [00:00<00:00, 4904698.80it/s]

100%|███████████████████████████████████████| 15/15 [00:00<00:00, 393216.00it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 29746.84it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 74235.47it/s]

100%|████████████████████████████████████████| 94/94 [00:00<00:00, 80874.78it/s]

100%|███████████

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 33288.13it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 29959.31it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38130.04it/s]

100%|███████████████████████████████████████| 10/10 [00:00<00:00, 352462.52it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 70492.50it/s]

100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 85598.04it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 29537.35it/s]

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 62137.84it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 34952.53it/s]

100%|█████████████████████████████████████████| 5/5 [00:00<00:00, 177724.75it/s]

100%|██████████████████████████████████████| 65/65 [00:00<00:00, 1323445.44it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 29330.80it/s]

100%|███████████

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38479.85it/s]

100%|█████████████████████████████████████████| 8/8 [00:00<00:00, 294337.12it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 39199.10it/s]

100%|██████████████████████████████████████| 90/90 [00:00<00:00, 1534501.46it/s]

100%|██████████████████████████████████████| 75/75 [00:00<00:00, 1883669.46it/s]

100%|██████████████████████████████████████| 46/46 [00:00<00:00, 1330606.79it/s]

100%|█████████████████████████████████████████| 7/7 [00:00<00:00, 225847.14it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38479.85it/s]

100%|█████████████████████████████████████████| 4/4 [00:00<00:00, 138654.68it/s]

100%|████████████████████████████████████████| 42/42 [00:00<00:00, 59094.52it/s]

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 38130.04it/s]

100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 111353.20it/s]

100%|███████████

In [19]:
len(all_samples)
len(all_labels)

681

In [20]:
len(all_samples[0])

38

In [18]:
import math

dummy_features = (math.floor(np.sqrt(38))+1)**2 - 38
print(dummy_features)
square_size = int(np.sqrt(dummy_features+38))
print(square_size)

11
7


In [30]:
xx = np.zeros((len(all_samples),38))# 1, square_size, square_size))
yy = np.zeros((len(all_samples), 2))

for i in range(len(all_samples)):
    #data = np.zeros(dummy_features+38)
    #data[:38] = all_samples[i]
    #data[38:] = np.zeros((dummy_features))
    #xx[i,:,:] = data.reshape(1, square_size, square_size)
    xx[i,:] = all_samples[i]
    yy[i,:] = all_labels[i]
    print(all_samples[i])
    
x_mean = np.mean(xx, axis=0)
y_mean = np.mean(yy, axis=0)
xx = xx-x_mean
yy = yy-y_mean

[0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 15.0, 0.0, 1.0, 0.0, 0.0, 0.0, 82.0, 0.0, 32.0, 0.0, 21.0, 0.0, 190.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 733]
[0.0, 1.0, 0.0, 25.0, 0.0, 212.0, 0.0, 36.0, 0.0, 1009.0, -0.06250000000000006, 34962.0, 0.03333333333333337, 4928.0, 0.03225806451612906, 17660.0, 0.0, 16149.0, 0.0, 39908.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 733]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 733]
[0.0, 50.0, 0.0, 183.0, 0.0, 192.0, 0.0, 585.0, 0.0, 1031.0, 0.0, 851.0, 0.0, 4072.0, 0.0, 9794.0, 0.0, 1883.0, 0.0, 1627.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 733]
[0.0, 295.0, 0.0, 1864.0, 0.0, 1764.0, 0.0, 254.0, 0.0, 208.0, 0.0, 610.0, 0.0, 329.0, 0.0, 10214.0, 0.0, 10521.0, 0.0, 82865.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 733]
[0.4000000000000001, 2574.0, -0.28571428571428575, 7466.0, 4.25, 516.0, -0.838095

In [31]:
np.save(os.getcwd() + "/data/xx-38.npy", xx)
np.save(os.getcwd() + "/data/yy-38.npy", yy)

In [32]:
x_mean.shape

(38,)